In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/

In [2]:
!pip install cloud-volume
!pip install -e git+https://github.com/seung-lab/tqdm.git#egg=tqdm

nbformat 4.4.0 has requirement jsonschema!=2.5.0,>=2.4, but you'll have jsonschema 2.3.0 which is incompatible.
nbconvert 5.3.1 has requirement pandocfilters>=1.4.1, but you'll have pandocfilters 1.2 which is incompatible.
google-api-core 0.1.4 has requirement setuptools>=34.0.0, but you'll have setuptools 20.7.0 which is incompatible.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Obtaining tqdm from git+https://github.com/seung-lab/tqdm.git#egg=tqdm
  Updating ./src/tqdm clone
nbconvert 5.3.1 has requirement pandocfilters>=1.4.1, but you'll have pandocfilters 1.2 which is incompatible.
nbformat 4.4.0 has requirement jsonschema!=2.5.0,>=2.4, but you'll have jsonschema 2.3.0 which is incompatible.
google-api-core 0.1.4 has requirement setuptools>=34.0.0, but you'll have setuptools 20.7.0 which is incompatible.
  Found existing installation: tqdm 4.23.0
    Can't uninstall 'tqdm'. No files

In [3]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F 
import torch.optim as optim

from skimage.transform import resize
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox, Vec
import numpy as np
import pandas as pd

In [4]:
# train utils
def np2var(arr, cuda):
    while len(arr.shape) < 4:
        arr = np.expand_dims(arr, 0)
    arr = torch.from_numpy(arr)
    arr = Variable(arr)
    if cuda:
        arr = arr.cuda()
    return arr

def var2np(arr):
    arr = arr.data.cpu().numpy()
    while len(arr.shape) > 2:
        arr = arr[0]
    return arr

In [5]:
label_ids = np.array([1,2])

In [6]:
# helper operations 
def conv3x3(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels,
        kernel_size=3, stride=1, padding=1, bias=True)

def maxpool2x2():
    return nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

class UpConv2x2(nn.Module):
    def __init__(self, channels):
        super(UpConv2x2, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2)
        self.conv = nn.Conv2d(channels, channels // 2,
            kernel_size=2, stride=1, padding=0, bias=True)

    def forward(self, x):
        x = self.upsample(x)
        x = F.pad(x, (0,1,0,1))
        x = self.conv(x)
        return x 
        
def concat(xh, xv):
    return torch.cat([xh, xv], dim=1)


# unet blocks
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(ConvBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class DownConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(DownConvBlock, self).__init__()
        self.maxpool = maxpool2x2()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = self.maxpool(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UpConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(UpConvBlock, self).__init__()
        self.upconv = UpConv2x2(in_channels)
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, xh, xv):
        """
        Args:
            xh: torch Variable, activations from same resolution feature maps (gray arrow in diagram)
            xv: torch Variable, activations from lower resolution feature maps (green arrow in diagram)
        """
        xv = self.upconv(xv)
        x = concat(xh, xv)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        fs = [16,32,64,128,256]
        self.conv_in = ConvBlock(1, fs[0])
        self.dconv1 = DownConvBlock(fs[0], fs[1])
        self.dconv2 = DownConvBlock(fs[1], fs[2])
        self.dconv3 = DownConvBlock(fs[2], fs[3])
        self.dconv4 = DownConvBlock(fs[3], fs[4])

        self.uconv1 = UpConvBlock(fs[4], fs[3])
        self.uconv2 = UpConvBlock(fs[3], fs[2])
        self.uconv3 = UpConvBlock(fs[2], fs[1])
        self.uconv4 = UpConvBlock(fs[1], fs[0])
        self.conv_out = conv3x3(fs[0], len(label_ids))

        self._initialize_weights()

    def forward(self, x):
        x1 = self.conv_in(x)
        x2 = self.dconv1(x1)
        x3 = self.dconv2(x2)
        x4 = self.dconv3(x3)
        x5 = self.dconv4(x4)
        x6 = self.uconv1(x4, x5)
        x7 = self.uconv2(x3, x6)
        x8 = self.uconv3(x2, x7)
        x9 = self.uconv4(x1, x8)
        x10 = self.conv_out(x9)
        return x10

    def _initialize_weights(self):
        conv_modules = [m for m in self.modules() if isinstance(m, nn.Conv2d)]
        for m in conv_modules:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, np.sqrt(2. / n))


In [67]:
net = torch.load('basil_defect_unet_mip518070305')

In [68]:
def detect_defects_chunk(image_cv, fold_cv, crack_cv, full_dst_bbox, full_dst_padding):
    print(full_dst_bbox)
    full_src_bbox = Bbox(full_dst_bbox.minpt - full_dst_padding, full_dst_bbox.maxpt + full_dst_padding)
    src_bbox = Bbox.from_slices(image_cv.slices_from_global_coords(full_src_bbox.to_slices()))
    dst_bbox = Bbox.from_slices(image_cv.slices_from_global_coords(full_dst_bbox.to_slices()))
    
    img = image_cv[src_bbox.to_slices()][:,:,0,0].astype(np.float32)
    img = img - np.mean(img) # normalize inputs
    img = img / (np.std(img) + 1e-6)
    img = np2var(img, cuda=True)
    pred = net(img)
    
    local_bbox = dst_bbox - src_bbox.minpt
    defects = pred.squeeze(0)
    fold = (var2np(F.sigmoid(defects[0]))*255).astype(np.uint8)[local_bbox.to_slices()[:2]]
    crack = (var2np(F.sigmoid(defects[1]))*255).astype(np.uint8)[local_bbox.to_slices()[:2]]
    fold_cv[dst_bbox.to_slices()] = np.expand_dims(np.expand_dims(fold, 2), 3)
    crack_cv[dst_bbox.to_slices()] = np.expand_dims(np.expand_dims(crack, 2), 3)

In [69]:
def detect_defects(image_cv, fold_cv, crack_cv, bbox, dst_chunk, dst_padding):
    """
    bbox: Bbox @ mip0
    dst_chunk: Vec @ mip0
    dst_padding: Vec @ mip0
    """
    src_chunk = dst_chunk + dst_padding
    bbox = bbox.expand_to_chunk_size(fold_cv.underlying, fold_cv.voxel_offset)
    chunks = bbox / dst_chunk
    for x in range(chunks.size3()[0]):
        for y in range(chunks.size3()[1]):
            dst_origin = bbox.minpt + dst_chunk*Vec(x,y,1)
            dst_bbox = Bbox(dst_origin, dst_origin + dst_chunk)
            # print(dst_bbox)
            detect_defects_chunk(image_cv, fold_cv, crack_cv, dst_bbox, dst_padding)
    

In [70]:
image_cv_path = 'gs://neuroglancer/basil_v0/father_of_alignment/v3'
fold_cv_path = 'gs://neuroglancer/basil_v0/father_of_alignment/v3/mask/defect_detector_folds_v6'
crack_cv_path = 'gs://neuroglancer/basil_v0/father_of_alignment/v3/mask/defect_detector_cracks_v6'
image_cv = CloudVolume(image_cv_path, mip=5, bounded=False, fill_missing=True, parallel=1, progress=False)
info = image_cv.info
fold_cv = CloudVolume(fold_cv_path, mip=5, cdn_cache=False, fill_missing=True, info=info, non_aligned_writes=True, parallel=1, progress=False)
fold_cv.commit_info()
crack_cv = CloudVolume(crack_cv_path, mip=5, cdn_cache=False, fill_missing=True, info=info, non_aligned_writes=True, parallel=1, progress=False)
crack_cv.commit_info()
bbox = Bbox([131982, 99399, 180], [195807, 130501, 181])
# bbox = Bbox([9071, 10780, 180], [211838, 269420, 181])
# bbox = Bbox([6490, 2759, 690], [228902, 271673, 691])
dst_chunk = Vec(4096, 4096, 1)
dst_padding = Vec(512, 512, 0)
detect_defects(image_cv, fold_cv, crack_cv, bbox, dst_chunk, dst_padding)

Bbox([131008, 99264, 181],[135104, 103360, 182])
Bbox([131008, 103360, 181],[135104, 107456, 182])
Bbox([131008, 107456, 181],[135104, 111552, 182])
Bbox([131008, 111552, 181],[135104, 115648, 182])
Bbox([131008, 115648, 181],[135104, 119744, 182])
Bbox([131008, 119744, 181],[135104, 123840, 182])
Bbox([131008, 123840, 181],[135104, 127936, 182])
Bbox([135104, 99264, 181],[139200, 103360, 182])
Bbox([135104, 103360, 181],[139200, 107456, 182])
Bbox([135104, 107456, 181],[139200, 111552, 182])
Bbox([135104, 111552, 181],[139200, 115648, 182])
Bbox([135104, 115648, 181],[139200, 119744, 182])
Bbox([135104, 119744, 181],[139200, 123840, 182])
Bbox([135104, 123840, 181],[139200, 127936, 182])
Bbox([139200, 99264, 181],[143296, 103360, 182])
Bbox([139200, 103360, 181],[143296, 107456, 182])
Bbox([139200, 107456, 181],[143296, 111552, 182])
Bbox([139200, 111552, 181],[143296, 115648, 182])
Bbox([139200, 115648, 181],[143296, 119744, 182])
Bbox([139200, 119744, 181],[143296, 123840, 182])
Bbo